In [1]:
# Installing initial packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import awswrangler as wr
import boto3

import json
import io               
import os
from pandas import DataFrame
import datetime as dt
def install(package):
    subprocess.check_call([sys.executable, "-q", "-m", "pip", "install", package])


from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = 'all'

session = boto3.Session(region_name="ap-southeast-1")
pd.options.display.max_columns=None
pd.options.display.max_rows=None

In [2]:
def transform_df(members,claims_df,elig,yellow_members):
    
        ### Adjust diagnosis field ###
        # claims subset data
        
        members['diagnosis_key_for_lookup_1'] = members['diagnosis_key'].str.replace('.', '')
        members['diagnosis_key_for_lookup_2'] = members['diagnosis_key_for_lookup_1'].apply('{:0<5}'.format)
        
        
           
        ### Identify the MSK patients ###
        members2=members[members['diagnosis_key_for_lookup_2'].isin(['71516','72210','72220','72420','72430','72840','83600','83620','95920'])].groupby(['member_id']
        ).agg(
        members_distinct_count=('member_id', pd.Series.nunique),
        ).reset_index()
        
        #================================================================================
        
        
        
        ### Merging of claims subset data with complate claims data  ###
        
        claims_data=members2[['member_id']].merge(claims_df,how ='inner',left_on=['member_id'], right_on=['member_id'])
                
        
        ### Check for the distinct incurred_yrmth ###
        claims_data['incurred_yrmth'].value_counts().sort_index()
        
        
        #====================================================================================
        
        # merge eligibity data with claims subset-members2
              
        ### Merge eligibility data for MSK patients ###
        
        elig_data=members2[['member_id']].merge(elig,how ='inner',left_on=['member_id'], right_on=['member_id'])
        
        ### Checking for thr distribution of eligible year month ###
        elig_data['eligible_yrmth'].value_counts().sort_index()
        
        
        
        
        
        
        
        #====================================================================================
        
        
        ### Link eligibility data to entire claims data ###
        
        elig_claim=elig_data.merge(claims_data,how ='left',left_on=['member_id', 'eligible_yrmth','group_id'], right_on=['member_id','incurred_yrmth','group_id'])
        
        ### Drop redundent fields ###
        elig_claim.drop(['member_city_y','group_name_y','plan_code_y','subscriber_no_y','client_name_y','funding_type_y','subscriber_type_y','member_location_y','person_no_y','product_id_y','member_gender_y','member_nationality_y','member_dob_y'], axis=1, inplace=True)
        elig_claim = elig_claim.rename(columns={'plan_code_x':'plan_code','member_gender_x':'member_gender','group_name_x':'group_name','subscriber_no_x':'subscriber_no','funding_type_x':'funding_type','member_nationality_x':'member_nationality','person_no_x':'person_no','member_location_x':'member_location','member_city_x':'member_city','subscriber_type_x':'subscriber_type','client_name_x':'client_name','product_id_x':'product_id','member_dob_x':'member_dob'})


        ### Unwanted columns removed and drop duplicates ###
        elig_claim_v2 = elig_claim.drop_duplicates()
        
        ### Minor Exlusion from the analysis ###
        msk_elig_claim = elig_claim_v2.loc[(elig_claim_v2['member_age'] >= 18)]
       
      

        ## Deriving a column visits_derived from incurred_date and member_id ###
        msk_elig_claim['incurred_date'] = pd.to_datetime(msk_elig_claim['incurred_date'])
        msk_elig_claim['visits_derived'] = msk_elig_claim['member_id'].astype(str)+'_'+ msk_elig_claim['incurred_date'].dt.date.astype(str)
        
        
        
        
        del members
        del members2
        del elig_data
        del elig_claim
        del elig_claim_v2
        
        #=================================================================================
        #====================================================================================
        
        
        # Observation period 1 year - date automated
        ### Extract the observation period ###
        
        #observation period:example #example: model run perio 28jan21-28dec2023, 
        #index date=28 sept 2023
        #observation period: 28sept2022-28sep2023      
        
        
        
        
        
        
        msk_elig_claim_obs=msk_elig_claim.loc[(msk_elig_claim['incurred__date__year_month_']>=(msk_elig_claim['incurred__date__year_month_'].max()
                                                                                               -pd.DateOffset(months=15)))
                   & (msk_elig_claim['incurred__date__year_month_']<=(msk_elig_claim['incurred__date__year_month_'].max()-
                                                                     pd.DateOffset(months=3))) ]
        
                        
        # Adding manipulated diagnosis_key column #
        msk_elig_claim_obs['diagnosis_key_for_lookup_1'] = msk_elig_claim_obs['diagnosis_key'].str.replace('.', '')
        msk_elig_claim_obs['diagnosis_key_for_lookup_2'] = msk_elig_claim_obs['diagnosis_key_for_lookup_1'].apply('{:0<5}'.format)

        
        
        #===========================================================================================
        ### Getting eligible month member count ###
        elig_member_count_df = msk_elig_claim_obs.groupby(['member_id']).agg(
            eligible_yrmth_count=('eligible_yrmth', 'nunique')
        )
        elig_member_count_df = elig_member_count_df.reset_index()
                     
          
    
    
        #===============================================================================================
        
        ### Processing member level data with total paid amount and visit_count summed ### ! Make sure to name visits_counts as total_visits_count
        grouped_msk = msk_elig_claim_obs.groupby(['member_id', 'market','member_location','member_age','member_gender','claim_type',]).agg(
            total_paid_amount=('gross_paid_amount', 'sum'),
            visits_count=('visits_derived', 'nunique')
        )
        grouped_msk = grouped_msk.reset_index()
        
        
  
        #============================================================================================
    
        ### Merge the eligible month count dataframe and grouped dataframe ###
        elig_grouped_msk=pd.merge(grouped_msk, elig_member_count_df, left_on='member_id', right_on='member_id', how='inner')

        duplicate_count = elig_grouped_msk.duplicated('member_id').sum()
        
        
        #==============================================================================================
        #=============================================================================================
        del elig_member_count_df
        del grouped_msk
        del duplicate_count
        #=============================================================================================
        
        
        
        

        #### Implementing logic to market with highest health care visits affliated to a member ###
        # Calculate the total_visits counts for each member and market
        total_visits = elig_grouped_msk.groupby(['member_id','market','eligible_yrmth_count'])['visits_count'].sum().reset_index()

        # Find the market with highest visit care counts for each member
        highest_visit_market = total_visits.groupby('member_id')['visits_count'].idxmax()
        # highest_visit_market.head()

        #Create a new dataframe from selected market values and total_visits
        msk_claim_visits_df = total_visits.loc[highest_visit_market,['member_id','market','visits_count','eligible_yrmth_count']]
        msk_claim_visits_df.reset_index(drop= True, inplace=True)

        ### Distinguish OP and IP claim total paid amount ###
        msk_claims_amt_df = elig_grouped_msk.pivot_table('total_paid_amount',['member_id','member_location','member_age','member_gender'],'claim_type')

        # Removing the indexing and renaming the axis
        msk_claims_amt_df.reset_index(inplace=True)
        msk_claims_amt_df = msk_claims_amt_df.rename_axis(None,axis=1)
        column_mapping = {'IP':'IP_total_paid_amount','OP':'OP_total_paid_amount'}
        msk_claims_amt_df.rename(columns=column_mapping,inplace=True)

        # Fill NaN values with 0 
        msk_claims_amt_df['IP_total_paid_amount'].fillna(0,inplace=True)
        msk_claims_amt_df['OP_total_paid_amount'].fillna(0,inplace=True)
        

        ### Distincguish OP and IP visits ###
        msk_claims_visits_df = elig_grouped_msk.pivot_table('visits_count',['member_id','member_location','member_age','member_gender'],'claim_type')

        # Removing the indexing and renaming the axis
        msk_claims_visits_df.reset_index(inplace=True)
        msk_claims_visits_df = msk_claims_visits_df.rename_axis(None,axis=1)
        column_mapping = {'IP':'IP_visits_count','OP':'OP_visits_count'}
        msk_claims_visits_df.rename(columns=column_mapping,inplace=True)

        # Fill NaN values with 0 
        msk_claims_visits_df['IP_visits_count'].fillna(0,inplace=True)
        msk_claims_visits_df['OP_visits_count'].fillna(0,inplace=True)
        
        
        
        
        
        ### Create a functionto check duplicacy ###
        def duplicacy(df):
            duplicate_member_ids = df['member_id'].duplicated()
            if duplicate_member_ids.any():
                print("Duplicate member_ids found")
                duplicate_rows = df[duplicate_member_ids]
                print(duplicate_rows)
            else:
                print("No duplicate member_ids found.")
                
        
        
        ### Get distinguished IP/OP visits and total_paid_amount ###
        msk_claims_final=msk_claims_visits_df[['member_id','member_location','member_age','member_gender','IP_visits_count','OP_visits_count']].merge(msk_claims_amt_df[['member_id','IP_total_paid_amount','OP_total_paid_amount']],how ='inner',left_on=['member_id'], right_on=['member_id'])
        
        

        msk_claims_final=msk_claim_visits_df.merge(msk_claims_final,how ='inner',left_on=['member_id'], right_on=['member_id'])

        # Renaming the market to relevant column name --- {market : market_most_visits} #
        msk_claims_final.rename(columns={'market':'market_most_visits'},inplace=True)


        
        # Obtain unqiue members in the dataset #
        msk_claims_final['member_id'].nunique() #--- # Matches with rows of the dataset!!!
        
        
        #==================================================================================================
        
        
        
        del msk_claims_visits_df
        del msk_claims_amt_df
        
        
        
        #==================================================================================================
        
        #Adjust diagnosis field
        yellow_members['diagnosis_key_for_lookup_1'] = yellow_members['diagnosis_key'].str.replace('.', '')
        yellow_members['diagnosis_key_for_lookup_2'] = yellow_members['diagnosis_key_for_lookup_1'].apply('{:0<5}'.format)
        yellow_members['incurred_date'] = pd.to_datetime(yellow_members['incurred_date'])
        yellow_members['visits_derived'] = yellow_members['member_id'].astype(str)+'_'+ yellow_members['incurred_date'].dt.date.astype(str)

        
        
        def yellowFlagging_type1(df, df2, in_list, indicator_name,  indicator_agg_col):
            
            members_yf=df[df['diagnosis_key_for_lookup_2'].isin(in_list)].groupby(['member_id']
            ).agg(
            aggregator_col_name = ('visits_derived',pd.Series.nunique)
            ).reset_index()

            df2[indicator_name] =  df2["member_id"].isin(members_yf["member_id"]).astype(int)
            
            # Joining the tables to get the yellow flag visits column
            df2=df2.merge(members_yf,how ='left',on=['member_id'])

            df2.rename(columns={'aggregator_col_name': indicator_agg_col}, inplace=True)

            df2[indicator_agg_col].fillna(0,inplace=True)
            
            return df2
        
        


        def yellowFlagging_type2(df, df2, in_list, indicator_name,  indicator_agg_col):
            
            members_yf=df[df['diagnosis_key_for_lookup_2'].str.startswith(tuple(in_list))].groupby(['member_id']
            ).agg(
            aggregator_col_name = ('visits_derived',pd.Series.nunique)
            ).reset_index()


            df2[indicator_name] =  df2["member_id"].isin(members_yf["member_id"]).astype(int)
            
            # Joining the tables to get the yellow flag visits column
            df2=df2.merge(members_yf,how ='left',on=['member_id'])

            df2.rename(columns={'aggregator_col_name': indicator_agg_col}, inplace=True)         

            df2[indicator_agg_col].fillna(0,inplace=True)
            
            return df2
        
        
        
        

        ### Create a function to compute the visit rate per 1000
        def visit_rate_compute(df,visits_rate,visits_count):
            df[visits_rate]=(df[visits_count]/df['eligible_yrmth_count'])*1000
            return df
        
        def visit_rate_compute_v2(df,visits_rate,visits_count):
            df[visits_rate]=(df[visits_count]/df['eligible_yrmth_count'])
            return df
        
        
        msk_claims_final = yellowFlagging_type1(yellow_members, msk_claims_final,['24900','24901','24960','24961','24970','24971','24980','24981','24990','24991'], 'yf249_Type1Diabetes_indicator','yf249_Type1Diabetes_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf249_Type1Diabetes_visits_rate','yf249_Type1Diabetes_visits_count')
        ##
        msk_claims_final = yellowFlagging_type1(yellow_members, msk_claims_final ,['25000','25002','25060','25062','25070','25080','25082','25090','25092'],'yf250_Type2Diabetes_indicator','yf250_Type2Diabetes_visits_count'  )
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf250_Type2Diabetes_visits_rate','yf250_Type2Diabetes_visits_count')
        ##
        msk_claims_final = yellowFlagging_type1(yellow_members, msk_claims_final , ['27200','27210','27220','27230','27230','27240','27250','27260','27270','27280','27290'],'yf272_Dyslipidemia_indicator','yf272_Dyslipidemia_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf272_Dyslipidemia_visits_rate','yf272_Dyslipidemia_visits_count')
        ##
        msk_claims_final = yellowFlagging_type1(yellow_members, msk_claims_final , ['40100','40110','40190','40200','40300','40400','40390'],'yf401_Hypertension_indicator','yf401_Hypertension_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf401_Hypertension_visits_rate','yf401_Hypertension_visits_count')
        ##
        msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , ['3051'],'yf305_TobaccoUse_indicator','yf305_TobaccoUse_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf305_TobaccoUse_visits_rate','yf305_TobaccoUse_visits_count')
        ##
        msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , ['3055'],'yf3055_illicitDrugUse_indicator','yf3055_illicitDrugUse_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf3055_illicitDrugUse_visits_count','yf3055_illicitDrugUse_visits_count')
        ##
        msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , '3055','yf311_Depression_indicator','yf311_Depression_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf311_Depression_visits_rate','yf311_Depression_visits_count')
        ##
        msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , '300','yf300_Anxiety_indicator','yf300_Anxiety_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf300_Anxiety_visits_rate','yf300_Anxiety_visits_count')
        ##
        msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , '955','yf955_Polypharmcy_indicator','yf955_Polypharmcy_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf955_Polypharmcy_visits_rate','yf955_Polypharmcy_visits_count')
        ##
        msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , '278','yf278_Obesity_indicator','yf278_Obesity_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf278_Obesity_visits_rate','yf278_Obesity_visits_count')
        ##
        msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , '3002','yf3002_Kinesiophobia_indicator','yf3002_Kinesiophobia_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf3002_Kinesiophobia_visits_rate','yf3002_Kinesiophobia_visits_count')
        ##
        msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final ,['309','v60','z63'],'yf309_Acopia_indicator','yf309_Acopia_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf309_Acopia_visits_rate','yf309_Acopia_visits_count')
        ##
        
        
        
        #===================================================================================================
        ### Renaming and computing visit rate for respective yellow flags ###
        msk_claims_final.rename(columns={'visits_count':'total_visits_count' },inplace=True)
        # Adding Total gross paid amount column to the dataset
        msk_claims_final["total_paid_amount"] = msk_claims_final["IP_total_paid_amount"] + msk_claims_final["OP_total_paid_amount"]

        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'total_visits_rate','total_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'IP_total_visits_rate','IP_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'OP_total_visits_rate','OP_visits_count')
        
        
         ### Compute PPPM value for observation period ###
        msk_claims_final['pppm_obs'] = msk_claims_final['total_paid_amount']/msk_claims_final['eligible_yrmth_count']
        msk_claims_final['IP_total_paid_amount_pppm'] = msk_claims_final['total_paid_amount']/msk_claims_final['eligible_yrmth_count']
        msk_claims_final['OP_total_paid_amount_pppm'] = msk_claims_final['total_paid_amount']/msk_claims_final['eligible_yrmth_count']
        ### Cost indicator computation as well ###
        msk_claims_final['cost_indicator_obs'] = msk_claims_final['pppm_obs'].apply(lambda x: int(1) if x >= 1000 else int(0))
        
        #======================================================================================================
        
        #### Adding metabolic syndrome indicator and identifier ####
        ## Adding Elevated fasting plasma glucose ###
        msk_claims_final = yellowFlagging_type1(yellow_members, msk_claims_final , ['79029'],'yf790_ElevatedPlasmaGlucose_indicator','yf790_ElevatedPlasmaGlucose_visits_count')
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf790_ElevatedPlasmaGlucose_visits_rate','yf790_ElevatedPlasmaGlucose_visits_count')

        ### Adding metabolic syndrome indetifier based on diagnosis code logic ###
        msk_claims_final = yellowFlagging_type2(yellow_members, msk_claims_final , ['2777'],'yf277_metabolicSyndrome_identifier','yf277_metabolicSyndrome_visits_count')
        ### Adding visits rate for metabolic syndrome ###
        msk_claims_final = visit_rate_compute_v2(msk_claims_final,'yf277_metabolicSyndrome_visits_rate','yf277_metabolicSyndrome_visits_count')

        # Create a new column for metabolic syndrome indicator and initialise it with 0
        msk_claims_final["yf_metabolicSyndrome_indicator"] = 0
        # Check if Metabolic Identifier is True
        # Count the number of True indicators
        trueIndicator_count = msk_claims_final[["yf401_Hypertension_indicator","yf278_Obesity_indicator","yf790_ElevatedPlasmaGlucose_indicator","yf272_Dyslipidemia_indicator"]].sum(axis=1)

        # Set the metabolic Syndrome indicator to 1 where Metabolic Indicator is True and atleast 3 other indicators are True
        msk_claims_final.loc[(trueIndicator_count>=3),"yf_metabolicSyndrome_indicator"] =1 

        #============================================================================================
        
        
        ### Adding all yellow flag indicator column inplace ###
        msk_claims_final['total_yf_indicator'] = msk_claims_final[["yf249_Type1Diabetes_indicator","yf250_Type2Diabetes_indicator","yf272_Dyslipidemia_indicator","yf401_Hypertension_indicator","yf305_TobaccoUse_indicator","yf311_Depression_indicator","yf300_Anxiety_indicator","yf955_Polypharmcy_indicator","yf309_Acopia_indicator","yf278_Obesity_indicator","yf3002_Kinesiophobia_indicator"]].max(axis=1)
        
        
        
        def pt_procedures_type1(df, df2, procedure_str, indicator_name,indicator_agg_col):                   

            members_yf=df[df['procedure_desc'].astype(str).str.contains(procedure_str)].groupby(['member_id']
            ).agg(
            aggregator_col_name = ('visits_derived',pd.Series.nunique)
            ).reset_index()

            df2[indicator_name] =  df2["member_id"].isin(members_yf["member_id"]).astype(int)

            # Joining the tables to get the yellow flag visits column
            df2=df2.merge(members_yf,how ='left',on=['member_id'])
            print("columns " , df2.columns)
            df2.rename(columns={'aggregator_col_name': indicator_agg_col}, inplace=True)
            print("columns " , df2.columns)

            df2[indicator_agg_col].fillna(0,inplace=True)

            return df2
        
        
        
        msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"PHYSO|PHYSIO|G8090S",'physiotherapy_proc_indicator','physiotherapy_visits_count')
        msk_claims_final = visit_rate_compute(msk_claims_final,'physiotherapy_visits_rate','physiotherapy_visits_count')

        msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final,"ARTHRO|SURG",'surgery_proc_indicator','surgery_visits_count')
        msk_claims_final = visit_rate_compute(msk_claims_final,'surgery_visits_rate','surgery_visits_count')

        msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"X-RAY|SCAN|MRI|ULTRASOUND|MAMMO|TOMOGRAPHY|ANGIOGRAPHY|IMAGE",'radio_diag_scans_proc_indicator','radio_diag_scans_visits_count')
        msk_claims_final = visit_rate_compute(msk_claims_final,'radio_diag_scans_visits_rate','radio_diag_scans_visits_count')

        msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"CONSULT|SPCAN|PHYSICIAN|PHYSICAL|PLANNING|EVAL|VIST|EXAM|SCREEN|COUNSEL|ROUTINE",'eval_manag_services_proc_indicator','eval_manag_services_visits_count')
        msk_claims_final = visit_rate_compute(msk_claims_final,'eval_manag_services_visits_rate','eval_manag_services_visits_count')

        msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"ALTERNATIVE|CHINESE|ACUPUNCTURE|HERBAL|MASSAGE|AYUR|OSTEOPATHY",'alternative_medicine_proc_indicator','alternative_medicine_visits_count')
        msk_claims_final = visit_rate_compute(msk_claims_final,'alternative_medicine_visits_rate','alternative_medicine_visits_count')

        msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"DRUG|RX|PHARMA",'drugs_related_proc_indicator','drugs_related_visits_count')
        msk_claims_final = visit_rate_compute(msk_claims_final,'drugs_related_visits_rate','drugs_related_visits_count')

        msk_claims_final = pt_procedures_type1(msk_elig_claim_obs, msk_claims_final ,"ANAES|SEDATION",'anesthesia_proc_indicator','anesthesia_visits_count')
        msk_claims_final = visit_rate_compute(msk_claims_final,'anesthesia_visits_rate','anesthesia_visits_count')
        
        
        
        
        #### Appending the required demographic variables in the dataset ####

        # Getting the demographics variables Testing in progress
        msk_elig_claim_obs['incurred_date'] = pd.to_datetime(msk_elig_claim_obs['incurred_date'])

        # Sort the dataset in descending order based on incurred_date
        msk_elig_claim_obs_sorted = msk_elig_claim_obs.sort_values(by="incurred_date",ascending=False)

        # Create a new dataset by grouping and applying the lambda function
        msk_elig_claim_obs_v2 = msk_elig_claim_obs_sorted.groupby('member_id').apply(lambda x:x.head(1).reset_index(drop=True))

        msk_elig_claim_obs_subset = msk_elig_claim_obs_v2[['member_id','group_industry_sector_desc','member_family_type','member_nationality','client_location','market']]
        msk_elig_claim_obs_subset.rename(columns={'market':'market_latest_index' },inplace=True)
        msk_elig_claim_obs_subset.reset_index(drop=True,inplace=True)

        ### Merge and append the attributes for demographic variables based on latest index on visits ###
        msk_claims_final = msk_claims_final.merge(msk_elig_claim_obs_subset,how ='left',on=['member_id'])
        
        
        
        
        ### Mapping industry ###
        industry_mapping = {'Food & Textiles':'Food Textiles and Retailing',
                      'Food Retailing': 'Food Textiles and Retailing' ,
                       'Miscellaneous': 'Miscellaneous',
                      'Miscellaneous Services':'Miscellaneous' ,
                       'Other Manufacturing':'Manufacturing and Retail',
                       'Other Retail': 'Manufacturing and Retail',
                       'Passenger Transportation':'Transportation and Services',
                       'Transport Services':'Transportation and Services'}

        def map_to_new_category(old_industry):
            return industry_mapping.get(old_industry,old_industry)

        msk_claims_final['member_industry'] = msk_claims_final['group_industry_sector_desc'].apply(map_to_new_category)
        #msk_claims_final['member_industry'].value_counts()
        
        
        
        
        #=========================================================================================================
        #==========================================================================================================
        
        ## read country reference file from s3
        
        
        ### Mapping countries to respective regions ###
        df_country_region = pd.read_csv('s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/countryISOreference.csv',encoding='ISO-8859-1')
        country_to_region_dict = dict(zip(df_country_region['countryShortNameISO'],df_country_region['Region']))

        capitalize_country_to_region_dict = {(k.upper() if isinstance(k,str) else k):
                                             (v.upper() if isinstance (v,str) else v)
                                             for k,v in country_to_region_dict.items()}

        country_to_region_dict = capitalize_country_to_region_dict
        
        
        
        #=========================================================================================================
        
        
        !pip install fuzzywuzzy

        from fuzzywuzzy import fuzz 
        from fuzzywuzzy import process

        def find_best_match(member_location):
            best_match, score = process.extractOne(member_location,country_to_region_dict.keys())

            if score > 80 :
                return country_to_region_dict[best_match]
            else:
                return member_location

        msk_claims_final['member_location_region'] = msk_claims_final['member_location'].apply(find_best_match)
        msk_claims_final['member_nationality_region'] = msk_claims_final['member_nationality'].apply(find_best_match)
        
        
        #======================================================================================================
        
        ##### Adding maternity indicator to the dataset

        def maternity_indicator(df,column_name,in_list,indicator_name,df2):
            mat_mem = df[df[column_name].isin(in_list)].groupby(['member_id']
            ).agg(
            members_distinct_count=('member_id', pd.Series.nunique),
            ).reset_index()

            df2[indicator_name] =  df2["member_id"].isin(mat_mem["member_id"]).astype(int)

            df2[indicator_name].value_counts()
            return df2

        msk_claims_final = maternity_indicator(msk_elig_claim_obs,'procedure_code',['R1820S','R1820A','R2490A','R2290S','BPUMP','MATGS'],'maternity_procedure_indicator',msk_claims_final)
        msk_claims_final = maternity_indicator(msk_elig_claim_obs,'case_type_0001',['CM','NM'],'maternity_case_type_indicator',msk_claims_final)
        #### Create a indicator for maternity members with diagnosis code ####
        mat_codes = [i for i in range(630,780)]
        mat_codes = [str(x*100) for x in mat_codes]

        msk_claims_final = maternity_indicator(msk_elig_claim_obs,'diagnosis_key_for_lookup_2',mat_codes,'maternity_diagnosis_indicator',msk_claims_final)

        #### Create all_maternity_indicator column ####
        msk_claims_final['all_maternity_indicator'] = msk_claims_final[["maternity_procedure_indicator","maternity_case_type_indicator","maternity_diagnosis_indicator"]].max(axis=1)

        
        
        
        
        #=============================================================================================================
        
        
        # performace period with date automated
        #### Extracting performance period of 3 months#
        
        #example: model run perio 28jan21-28dec2023, 
        #index date=28 sept 2023
        #observation period: 28sept2022-28sep2023
        #performace period: #29 sep 2023 -28 dec 2023
        
        #
        
        msk_elig_claims_perf=msk_elig_claim.loc[(msk_elig_claim['incurred__date__year_month_']>=(msk_elig_claim['incurred__date__year_month_'].max()
                                                                                                 -pd.DateOffset(months=3)
                                                                                                 +pd.DateOffset(days=1)))
                   & (msk_elig_claim['incurred__date__year_month_']<=(msk_elig_claim['incurred__date__year_month_'].max())) ] 
        
        
        
        elg_df_perf = msk_elig_claims_perf.groupby(['member_id']).agg(
            eligible_yrmth_count_perf=('eligible_yrmth', 'nunique'),
            total_paid_amount_perf=('gross_paid_amount', 'sum')
        )
        elg_df_perf = elg_df_perf.reset_index()
        

        elg_df_perf['pppm_perf'] = elg_df_perf['total_paid_amount_perf']/elg_df_perf['eligible_yrmth_count_perf']
        elg_df_perf['cost_indicator_perf'] = elg_df_perf['pppm_perf'].apply(lambda x: int(1) if x >= 1000 else int(0))
        
        
        

        msk_claims_final =msk_claims_final.merge(elg_df_perf,how ='left',on=['member_id'])
        
        # Manipulate the suggested columns to missing --- 04-12-2023
        values_to_replace = {'General Construction & Related Services':pd.NA}
        msk_claims_final['member_industry'] = msk_claims_final['member_industry'].replace(values_to_replace)
        
        
        
        # Manipulate the suggested columns to missing --- 04-12-2023----- (market_latest_index)
        values_to_replace = {'Singapore':'market1','Hong Kong':'market2'}
        msk_claims_final['market_latest_index'] = msk_claims_final['market_latest_index'].replace(values_to_replace)
        
        
        
        #=============================================================================================
        # Delete US nationals from the dataset
        msk_claims_final_v2 = msk_claims_final.drop(msk_claims_final[msk_claims_final['member_nationality'] == 'UNITED STATES'].index)
        
        
        
        
        # Manipulate the suggested columns to missing --- 04-12-2023
        values_to_replace = {'General Construction & Related Services':pd.NA}
        msk_claims_final['member_industry'] = msk_claims_final['member_industry'].replace(values_to_replace)
        
        
        # Manipulate the suggested columns to missing --- 04-12-2023----member_family_type
        values_to_replace = {'#':pd.NA}
        msk_claims_final_v2['member_family_type'] = msk_claims_final_v2['member_family_type'].replace(values_to_replace)
        
        #df_msk =msk_claims_final_v2
        df= msk_claims_final_v2
        
#         df2= msk_claims_final_v2
        
        
        
        #==========================================================================================================
        #===================================================================================================
#         #Sending MSK patient claims+eligibility dataframe from sagemaker to a S3 folder
#         wr.s3.to_csv(
#         df=msk_claims_final_v2,
#         path='s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/msk_global_patients_claims_elig_Abhiya_AUTO_V1.csv')
        
#         #import table
#         df = pd.read_csv('s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/msk_global_patients_claims_elig_Abhiya_AUTO_V1.csv')
        
        
        
        df2 = df.rename(columns={'physiotherapy_visits_rate': 'physiotherapy_visits_rate',
        #                          'surgery_visits_rate': 'surgery_proc_visits_rate',
                              'surgery_proc_visits_rate': 'surgery_visits_rate',
        #                          'radio_diag_scans_visits_rate': 'radio_diag_scans_proc_visits_rate',
                             'radio_diag_scans_proc_visits_rate': 'radio_diag_scans_visits_rate',
        #                          'eval_manag_services_visits_rate': 'eval_manag_services_proc_visits_rate',
                             'eval_manag_services_proc_visits_rate': 'eval_manag_services_visits_rate',
        #                          'alternative_medicine_visits_rate': 'alternative_medicine_visits_rate',
        #                          'drugs_related_visits_rate': 'drugs_related_visits_rate',
        #                          'anesthesia_visits_rate':'anesthesia_visits_rate',
        #                          'cost_indicator_2021':'cost_indicator_obs',  
        #                          'new_group_industry':'member_industry'
                            })
        
        
        
        
        
        cols_to_fill=['member_gender','member_family_type','market_latest_index','member_industry','member_location_region']
        #df[cols_to_fill]=df[cols_to_fill].fillna('')
        df2[cols_to_fill]=df2[cols_to_fill].fillna('')
        
        #del msk_claims_final_v2 
        #del msk_claims_final
        
        del df

        #==================================================================================================
        
        #manual encoding
        # 'member_gender' 
        ####
        def member_gender_F(x):
            if x['member_gender'] in['F']:return 1
            else: return 0 
        df2['member_gender_F']=df2.apply(member_gender_F, axis=1)
        #####
        def member_gender_M(x):
            if x['member_gender'] in['M']:return 1
            else: return 0 
        df2['member_gender_M']=df2.apply(member_gender_M, axis=1)


        # 'member_family_type'
        ####
        def member_family_type_C(x):
            if pd.isna(x['member_family_type']):return 0
            elif x['member_family_type'] in['C']:return 1
            else: return 0 
        df2['member_family_type_C']=df2.apply(member_family_type_C, axis=1)
        #####
        
        def member_family_type_D(x):
            if x['member_family_type'] in['D']:return 1
            else: return 0 
        df2['member_family_type_D']=df2.apply(member_family_type_D, axis=1)
        #####
        def member_family_type_F(x):
            if x['member_family_type'] in['F']:return 1
            else: return 0 
        df2['member_family_type_F']=df2.apply(member_family_type_F, axis=1)

        #####
        def member_family_type_S(x):
            if x['member_family_type'] in['S']:return 1
            else: return 0 
        df2['member_family_type_S']=df2.apply(member_family_type_S, axis=1)


        # 'market_latest_index'
        #####
        def market_latest_index_market1(x):
            if x['market_latest_index'] in['market1']:return 1
            else: return 0 
        df2['market_latest_index_market1']=df2.apply(market_latest_index_market1, axis=1)

        #####
        def market_latest_index_market2(x):
            if x['market_latest_index'] in['market2']:return 1
            else: return 0 
        df2['market_latest_index_market2']=df2.apply(market_latest_index_market2, axis=1)


        # 'member_industry'
        ####
        def member_industry_Banking_Investment_and_Insurance_inc_Brokers_Agent_(x):
            if x['member_industry'] in['Banking,Investment & Insurance (inc.Brokers,Agent)']:return 1
            else: return 0 
        # setattr(sys.modules[__name__], 'member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_', member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_)
        df2['member_industry_Banking_Investment_and_Insurance_inc_Brokers_Agent_']=df2.apply(member_industry_Banking_Investment_and_Insurance_inc_Brokers_Agent_, axis=1)
        df2.rename(columns={'member_industry_Banking_Investment_and_Insurance_inc_Brokers_Agent_':'member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_'}, inplace=True)
        #####
        def member_industry_Chemicals_Petroleum_Refining_and_Assoc_Products(x):
            if x['member_industry'] in['Chemicals, Petroleum Refining & Assoc.Products']:return 1
            else: return 0 
        df2['member_industry_Chemicals_Petroleum_Refining_and_Assoc_Products']=df2.apply(member_industry_Chemicals_Petroleum_Refining_and_Assoc_Products, axis=1)
        df2.rename(columns={'member_industry_Chemicals_Petroleum_Refining_and_Assoc_Products':'member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products'}, inplace=True)
        #####
        def member_industry_Communications_and_Computing(x):
            if x['member_industry'] in['Communications & Computing']:return 1
            else: return 0 
        df2['member_industry_Communications_and_Computing']=df2.apply(member_industry_Communications_and_Computing, axis=1)
        df2.rename(columns={'member_industry_Communications_and_Computing':'member_industry_Communications_&_Computing'}, inplace=True)
        #####
        def member_industry_Community_and_Social_Services(x):
            if x['member_industry'] in['Community & Social Services']:return 1
            else: return 0 
        df2['member_industry_Community_and_Social_Services']=df2.apply(member_industry_Community_and_Social_Services, axis=1)
        df2.rename(columns={'member_industry_Community_and_Social_Services':'member_industry_Community_&_Social_Services'}, inplace=True)
        ####
        def member_industry_Consultancy_Marketing_and_Legal_services(x):
            if x['member_industry'] in['Consultancy, Marketing & Legal services']:return 1
            else: return 0 
        df2['member_industry_Consultancy_Marketing_and_Legal_services']=df2.apply(member_industry_Consultancy_Marketing_and_Legal_services, axis=1)
        df2.rename(columns={'member_industry_Consultancy_Marketing_and_Legal_services':'member_industry_Consultancy_Marketing_&_Legal_services'}, inplace=True)
        #####
        def member_industry_Dev_Buying_and_Selling_of_Domestic_or_Comm_Property(x):
            if x['member_industry'] in['Dev, Buying & Selling of Domestic or Comm.Property']:return 1
            else: return 0 
        df2['member_industry_Dev_Buying_and_Selling_of_Domestic_or_Comm_Property']=df2.apply(member_industry_Dev_Buying_and_Selling_of_Domestic_or_Comm_Property, axis=1)
        df2.rename(columns={'member_industry_Dev_Buying_and_Selling_of_Domestic_or_Comm_Property':'member_industry_Dev_Buying_&_Selling_of_Domestic_or_Comm_Property'}, inplace=True)
        #####
        def member_industry_Education(x):
            if x['member_industry'] in['Education']:return 1
            else: return 0 
        df2['member_industry_Education']=df2.apply(member_industry_Education, axis=1)
        #####
        def member_industry_Electricity_and_Gas_Supply_Water_Sewage_and_Drainage(x):
            if x['member_industry'] in['Electricity & Gas Supply, Water, Sewage & Drainage']:return 1
            else: return 0 
        df2['member_industry_Electricity_and_Gas_Supply_Water_Sewage_and_Drainage']=df2.apply(member_industry_Electricity_and_Gas_Supply_Water_Sewage_and_Drainage, axis=1)
        df2.rename(columns={'member_industry_Electricity_and_Gas_Supply_Water_Sewage_and_Drainage':'member_industry_Electricity_&_Gas_Supply_Water_Sewage_&_Drainage'}, inplace=True)
        ####
        def member_industry_Food_Textiles_and_Retailing(x):
            if x['member_industry'] in['Food Textiles and Retailing']:return 1
            else: return 0 
        df2['member_industry_Food_Textiles_and_Retailing']=df2.apply(member_industry_Food_Textiles_and_Retailing, axis=1)
        #####
        def member_industry_Freight_Transportation_and_Warehousing(x):
            if x['member_industry'] in['Freight Transportation & Warehousing']:return 1
            else: return 0 
        df2['member_industry_Freight_Transportation_and_Warehousing']=df2.apply(member_industry_Freight_Transportation_and_Warehousing, axis=1)
        df2.rename(columns={'member_industry_Freight_Transportation_and_Warehousing':'member_industry_Freight_Transportation_&_Warehousing'}, inplace=True)
        #####
        def member_industry_Government_Administration(x):
            if x['member_industry'] in['Government Administration']:return 1
            else: return 0 
        df2['member_industry_Government_Administration']=df2.apply(member_industry_Government_Administration, axis=1)
        #####
        def member_industry_Health(x):
            if x['member_industry'] in['Health']:return 1
            else: return 0 
        df2['member_industry_Health']=df2.apply(member_industry_Health, axis=1)
        ####
        def member_industry_Hotels_and_Restaurants(x):
            if x['member_industry'] in['Hotels & Restaurants']:return 1
            else: return 0 
        df2['member_industry_Hotels_and_Restaurants']=df2.apply(member_industry_Hotels_and_Restaurants, axis=1)
        df2.rename(columns={'member_industry_Hotels_and_Restaurants':'member_industry_Hotels_&_Restaurants'}, inplace=True)
        #####
        def member_industry_Light_Electrical_and_Precision_Manufacturing(x):
            if x['member_industry'] in['Light, Electrical & Precision Manufacturing']:return 1
            else: return 0 
        df2['member_industry_Light_Electrical_and_Precision_Manufacturing']=df2.apply(member_industry_Light_Electrical_and_Precision_Manufacturing, axis=1)
        df2.rename(columns={'member_industry_Light_Electrical_and_Precision_Manufacturing':'member_industry_Light_Electrical_&_Precision_Manufacturing'}, inplace=True)
        #####
        def member_industry_Manufacturing_and_Retail(x):
            if x['member_industry'] in['Manufacturing and Retail']:return 1
            else: return 0 
        df2['member_industry_Manufacturing_and_Retail']=df2.apply(member_industry_Manufacturing_and_Retail, axis=1)
        #####
        def member_industry_Mining_and_Quarrying_incl_Oil_and_Gas_Extraction_(x):
            if x['member_industry'] in['Mining & Quarrying (incl. Oil & Gas Extraction)']:return 1
            else: return 0 
        df2['member_industry_Mining_and_Quarrying_incl_Oil_and_Gas_Extraction_']=df2.apply(member_industry_Mining_and_Quarrying_incl_Oil_and_Gas_Extraction_, axis=1)
        df2.rename(columns={'member_industry_Mining_and_Quarrying_incl_Oil_and_Gas_Extraction_':'member_industry_Mining_&_Quarrying_incl_Oil_&_Gas_Extraction_'}, inplace=True)
        #####
        def member_industry_Miscellaneous(x):
            if x['member_industry'] in['Miscellaneous']:return 1
            else: return 0 
        df2['member_industry_Miscellaneous']=df2.apply(member_industry_Miscellaneous, axis=1)
        #####
        def member_industry_Transportation_and_Services(x):
            if x['member_industry'] in['Transportation and Services']:return 1
            else: return 0 
        df2['member_industry_Transportation_and_Services']=df2.apply(member_industry_Transportation_and_Services, axis=1)
        #####
        def member_industry_Wholesale(x):
            if x['member_industry'] in['Wholesale']:return 1
            else: return 0 
        df2['member_industry_Wholesale']=df2.apply(member_industry_Wholesale, axis=1)    



        # 'member_location_region'
        ####
        def member_location_region_AFRICA(x):
            if x['member_location_region'] in['AFRICA']:return 1
            else: return 0 
        df2['member_location_region_AFRICA']=df2.apply(member_location_region_AFRICA, axis=1)
        #####
        def member_location_region_AMERICAS(x):
            if x['member_location_region'] in['AMERICAS']:return 1
            else: return 0 
        df2['member_location_region_AMERICAS']=df2.apply(member_location_region_AMERICAS, axis=1)
        #####
        def member_location_region_ASIA(x):
            if x['member_location_region'] in['ASIA']:return 1
            else: return 0 
        df2['member_location_region_ASIA']=df2.apply(member_location_region_ASIA, axis=1)

        #####
        def member_location_region_EUROPE(x):
            if x['member_location_region'] in['EUROPE']:return 1
            else: return 0 
        df2['member_location_region_EUROPE']=df2.apply(member_location_region_EUROPE, axis=1)

        #####
        def member_location_region_OCEANIA(x):
            if x['member_location_region'] in['OCEANIA']:return 1
            else: return 0 
        df2['member_location_region_OCEANIA']=df2.apply(member_location_region_OCEANIA, axis=1)


        #df2.head()
        
                
        #fillna for remainder fields
        df2[['member_age']]=df2[['member_age']].fillna(0).astype(np.int64)
        
        
 
        #final set of fields
        df3=df2[['member_id','member_age','IP_total_visits_rate', 'OP_total_visits_rate', 'yf249_Type1Diabetes_visits_rate', 'yf250_Type2Diabetes_visits_rate', 'yf272_Dyslipidemia_visits_rate', 'yf401_Hypertension_visits_rate', 'yf305_TobaccoUse_visits_rate', 'yf311_Depression_visits_rate', 'yf300_Anxiety_visits_rate', 'yf955_Polypharmcy_visits_rate', 'yf309_Acopia_visits_rate', 'yf278_Obesity_visits_rate', 'yf3002_Kinesiophobia_visits_rate', 'IP_total_paid_amount_pppm', 'OP_total_paid_amount_pppm', 'physiotherapy_visits_rate', 'surgery_visits_rate', 'radio_diag_scans_visits_rate', 'eval_manag_services_visits_rate', 'alternative_medicine_visits_rate', 'drugs_related_visits_rate', 'anesthesia_visits_rate', 'all_maternity_indicator', 'member_gender_F', 'member_gender_M', 'member_family_type_C', 'member_family_type_D', 'member_family_type_F', 'member_family_type_S', 'market_latest_index_market1', 'market_latest_index_market2', 'member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_', 'member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products', 'member_industry_Communications_&_Computing', 'member_industry_Community_&_Social_Services', 'member_industry_Consultancy_Marketing_&_Legal_services', 'member_industry_Dev_Buying_&_Selling_of_Domestic_or_Comm_Property', 'member_industry_Education', 'member_industry_Electricity_&_Gas_Supply_Water_Sewage_&_Drainage', 'member_industry_Food_Textiles_and_Retailing', 'member_industry_Freight_Transportation_&_Warehousing', 'member_industry_Government_Administration', 'member_industry_Health', 'member_industry_Hotels_&_Restaurants', 'member_industry_Light_Electrical_&_Precision_Manufacturing', 'member_industry_Manufacturing_and_Retail', 'member_industry_Mining_&_Quarrying_incl_Oil_&_Gas_Extraction_', 'member_industry_Miscellaneous', 'member_industry_Transportation_and_Services', 'member_industry_Wholesale', 'member_location_region_AFRICA', 'member_location_region_AMERICAS', 'member_location_region_ASIA', 'member_location_region_EUROPE', 'member_location_region_OCEANIA']]
        
        
        del df2
        
        # turn the memberid into an index
        final = df3.set_index('member_id')
        
        del df3 
        
        return final
    

    
    
if __name__ == "__main__":
        
    #     client = boto3.client('s3')
    #     parser = argparse.ArgumentParser() 
    #     parser.add_argument('--filename')
    #     args, _ = parser.parse_known_args()
    
    
    ### Extracting HK, SK between a data time frame needs to be specified
    ### Claims subset data - automated last two years

    query = """
    SELECT member_id,diagnosis_key from claims 
        where market in ('Singapore','Hong Kong') 
        and incurred__date__year_month_ >=(select
                                MAX(incurred__date__year_month_)-INTERVAL '2' year
                                from "hk_i_cleansed_ghb"."claims" )                           
    group by member_id,diagnosis_key
    """
    #setting the output of the query to a dataframe df, run this prior to running the df query
    members = wr.athena.read_sql_query(query,\
    #specify where the query runs                             
    database="hk_i_cleansed_ghb",\
    boto3_session=session,\
    ctas_approach=False,\
    workgroup='DataScientists')
    #view
    members.head()
    
    
    #----------
    #==============================================================================
    ## Two year entire Data from claims- last two years date automated
    
    
    
    query = """
    SELECT * FROM claims 
    where market in ('Singapore','Hong Kong') 
    and incurred__date__year_month_ >=(select
                                  MAX(incurred__date__year_month_)-INTERVAL '2' year 
                                  from "hk_i_cleansed_ghb"."claims")
    
     """
    claims_df = wr.athena.read_sql_query(query,\
    #specify where the query runs                             
    database="hk_i_cleansed_ghb",\
    boto3_session=session,\
    ctas_approach=False,\
    workgroup='DataScientists')
    #view
    claims_df.head()
    
    
    #----------------
    ##====================================================================================================
    
    ##Eligibility last two years - date automated
    ### Extracting of eligibilty data to get wide range of demographical attributes ### : (dates needs to mentioned)
    
    

    query = """
    SELECT 
    group_industry_sector_desc, plan_code, member_family_type, rider_5, type, member_gender, eap, group_name, subscriber_no, funding_type, group_id, member_nationality, vision, wellness, client_location, eligible_mth, dental, eligible_yrmth, person_no, member_location, member_city, aoc, eligible_yr, subscriber_type,
    member_age, policy_commencement, client_name, product_id, product_description,
    member_dob, member_id, subgroup_code
        from eligibility 
        where eligible_date >=(SELECT 
                    (MAX(eligible_date)- INTERVAL '2' year) as date_new
                     from "hk_i_cleansed_ghb"."eligibility")                
    group by
    group_industry_sector_desc, plan_code, member_family_type, rider_5, type, member_gender, eap, group_name, subscriber_no, funding_type, group_id, member_nationality, vision, wellness, client_location, eligible_mth, dental, eligible_yrmth, person_no, member_location, member_city, aoc, eligible_yr, subscriber_type,
    member_age, policy_commencement, client_name, product_id, product_description,
    member_dob, member_id, subgroup_code
    """
    #setting the output of the query to a dataframe df, run this prior to running the df query
    elig = wr.athena.read_sql_query(query,\
    #specify where the query runs                             
    database="hk_i_cleansed_ghb",\
    boto3_session=session,\
    ctas_approach=False,\
    workgroup='DataScientists')
    #view
    elig.head()
    
    
    
    #----------------
    #============================================================================================
    
    
    # Yellow flag members - data 2 years automated
    
    ### Create function for identifying the yellow flag members ###
    query = """
    SELECT member_id,diagnosis_key,incurred_date from claims 
        where market in ('Singapore','Hong Kong') 
        and incurred__date__year_month_ >=(select
                                MAX(incurred__date__year_month_)-INTERVAL '2' year
                                from "hk_i_cleansed_ghb"."claims")                         
    group by member_id,diagnosis_key,incurred_date
    """
    #setting the output of the query to a dataframe df, run this prior to running the df query
    yellow_members = wr.athena.read_sql_query(query,\
    #specify where the query runs                             
    database="hk_i_cleansed_ghb",\
    boto3_session=session,\
    ctas_approach=False,\
    workgroup='DataScientists')
    #view
    yellow_members.head()
    
    #--------------------------
    #==============================================================================================
    


,member_id,diagnosis_key
0,850772683:01,848.9
1,850779977:02,789.0
2,850779998:01,719.41
3,850698488:01,825.20
4,850688318:02,430


,member_age_band,paid_to,discharge_date,member_gender,incurred__date__year_month_,diag_classification_desc,procedure_desc,claimant_age,paid_date,plan_currency,country_of_treatment,claim_original_received_date,case_type_0001,incurreddate,not_covered_amount,case_type,withhold_amount,subscriber_no,fee_amount,ccs_lv1_desc,gross_paid_amount,ailment_desc,claim_type_key,copay_incurred_currency,plan_code,scale_of_cover,allowed_amount,client_name,lf_pay_sub_flag,provider_id,incurred_yrmth,visits,group_id,environment,diagnosis_key,claims_source,paid_yrmth,form_id,received_date,member_nationality,not_covered,clinks_claim,treatment_type,copay_amount,cci_chronic,partners,chronic_flag,incurred_currency,fallout_ind,member_termination_date,claim_detail_entered_date,denied_incurred_currency,subscriber_type,product_id,group_name,network,plan_desc,billed_amount,base_currency_discount,bed_nights,doctor_name,deductable_amount,ccs_lv2_desc,subgroup_id,member_commencement_date,deductable_incurred_currency,source,person_no,member_city,allowed_incurred_currency,provider_category,diagnosis_desc,member_dob,member_location,ccs_lv3_desc,fee_amount_paid_incurred_currency,funding_type,member_id,cl_not_cov_reason,provider_name,claim_type,procedure_code,cl_not_cov_reason_desc,market,fee_incurred_currency,billed_incurred_currency,paid_date__year_month_,net_amount,non_covered_reason_key,net_incurred_currency,fee_amount_paid,incurred_date,non_covered_reason_desc,region,sys_batchdttm
0,D,Paid To Member,1900-01-01,F,2023-12-21,Factors Influencing Health Status,OUT-PATIENT DRUGS,36.0,2023-12-27,SGD,SINGAPORE,2023-12-26,NM,20231221,0.0,Medical,0.0,850867321,0.000000,Complications of pregnancy; childbirth; and th...,60.596880,Under Investigation,OP,0.0,BWG,SGP,60.596880,FACEBOOK SINGAPORE PTE LTD,<NA>,SG0001,202312,1.0,607855,DL,V22,RI,202312,61190906,2023-12-26,SRI LANKA,0.0,N,Maternity,0.0,non-chronic condition,<NA>,N,SGD,NaN,1900-01-01,2023-12-27,0.0,Subscriber,SIMT,FACEBOOK SINGAPORE PTE LTD,OON,Facebook Singapore,60.596880,0.0,0.0,<NA>,0.0,Normal pregnancy and/or delivery,001,2021-10-01,0.0,Unknown,1,SINGAPORE,80.00,SG Others,NORMAL PREGNANCY,1987-02-27,SINGAPORE,Outcome of delivery (V codes),0.0,Fully Insured,850867321:01,<NA>,SINGAPORE,OP,OPDRG,<NA>,Singapore,0.0,80.00,2023-12-27,60.596880,@,80.00,0.0,2023-12-21,nan,Asia-Pacific,13
1,E,Paid To Provider,1900-01-01,M,2023-07-20,Factors Influencing Health Status,MEDICAL TAXABLE ITEMS,40.0,2023-12-19,SGD,SINGAPORE,2023-12-07,GM,20230720,0.0,Medical,0.0,850867321,0.000000,Symptoms; signs; and ill-defined conditions an...,14.285714,Under Investigation,OP,0.0,BWG,SGP,14.285714,FACEBOOK SINGAPORE PTE LTD,N,MPSHSI,202307,1.0,607855,DL,V72.6,CX,202312,61006288,2023-12-07,UNITED KINGDOM,0.0,Y,Pure Medical,0.0,non-chronic condition,Parkway,N,SGD,NaN,1900-01-01,2023-12-19,0.0,Spouse,SIMT,FACEBOOK SINGAPORE PTE LTD,In Network,Facebook Singapore,14.285714,0.0,0.0,<NA>,0.0,Factors influencing health care,001,2021-10-01,0.0,CIGNALINKS,2,SINGAPORE,18.86,SG Others,LABORATORY EXAMINATION,1983-07-01,SINGAPORE,Medical examination/evaluation,0.0,Fully Insured,850867321:02,<NA>,PARKWAYHEALTH LABORATORY (PARKWAY LABORATORY S...,OP,TAXEM,<NA>,Singapore,0.0,18.86,2023-12-19,14.285714,@,18.86,0.0,2023-07-20,nan,Asia-Pacific,13
2,E,Paid To Provider,1900-01-01,M,2023-07-20,Factors Influencing Health Status,PATHOLOGY - OUT-PATIENT,40.0,2023-12-19,SGD,SINGAPORE,2023-12-07,GM,20230720,0.0,Medical,0.0,850867321,9.922739,Symptoms; signs; and ill-defined conditions an...,188.532043,Under Investigation,OP,0.0,BWG,SGP,178.609304,FACEBOOK SINGAPORE PTE LTD,N,MPSHSI,202307,1.0,607855,DL,V72.6,CX,202312,61006288,2023-12-07,UNITED KINGDOM,0.0,Y,Pure Medical,0.0,non-chronic condition,Parkway,N,SGD,NaN,1900-01-01,2023-12-19,0.0,Spouse,SIMT,FACEBOOK SINGAPORE PTE LTD,In Network,Facebook Singapore,198.454782,26.2,0.0,<NA>,0.0,Factors influencing health care,001,2021-10-01,0.0,CIGNALINKS,2,SINGAPORE,235.80,SG Others,LABORATORY EXAMINATION,1983-07-01,SINGAP

,group_industry_sector_desc,plan_code,member_family_type,rider_5,type,member_gender,eap,group_name,subscriber_no,funding_type,group_id,member_nationality,vision,wellness,client_location,eligible_mth,dental,eligible_yrmth,person_no,member_location,member_city,aoc,eligible_yr,subscriber_type,member_age,policy_commencement,client_name,product_id,product_description,member_dob,member_id,subgroup_code
0,Communications & Computing,BWG,F,nan,TCN,F,Yes,FACEBOOK SINGAPORE PTE LTD,850891101,Fully Insured,607855,INDIA,No,No,Singapore,12,Yes,202312,4,SINGAPORE,SINGAPORE,SGP,2023,Dependant,8,2021-10-01,FACEBOOK SINGAPORE PTE LTD,SIMT,SGP REG MID TIER,2015-02-03,850891101:04,001
1,Miscellaneous Services,CVR,F,nan,KLN,M,No,GOOGLE ASIA PACIFIC PTE. LTD.,850891202,Fully Insured,611313,SINGAPORE,Yes,No,Singapore,12,Yes,202312,3,SINGAPORE,SINGAPORE,SGP,2023,Dependant,13,2023-05-01,GOOGLE SINGAPORE PTE LTD,SIMT,SGP REG MID TIER,2010-04-11,850891202:03,001
2,Communications & Computing,ADZ,F,nan,TCN,F,No,CTS SM & ABOVE PLAN 1 (WITH FLEX),850891351,Fully Insured,606834,PHILIPPINES,Yes,No,Hong Kong,12,Yes,202312,2,HONG KONG,TUNG CHUNG WATERFRONT RD,HKG,2023,Spouse,45,2020-12-01,COGNIZANT TECHNOLOGY SOLUTIONS (HK) LTD,HKMT,HK MED MID TIER,1978-01-21,850891351:02,001
3,Miscellaneous Services,DKC,F,nan,KLN,M,Yes,JANE STREET INTERNATIONAL,850891955,Fully Insured,605801,SINGAPORE,Yes,No,Hong Kong,12,Yes,202312,1,SINGAPORE,THE GLADES,W/W,2023,Employee,47,2020-01-01,JANE STREET ASIA LIMITED,HKME,HONG KONG MEDICAL,1976-10-02,850891955:01,002
4,Miscellaneous Services,DQD,F,nan,TCN,F,Yes,LEWIS SILKIN,850891992,Fully Insured,608646,UNITED KINGDOM,Yes,No,Hong Kong,12,Yes,202312,1,HONG KONG,HONG KONG,HKG,2023,Employee,51,2022-04-01,LEWIS SILKIN,HKMT,HK MED MID TIER,1972-04-26,850891992:01,001


,member_id,diagnosis_key,incurred_date
0,850867370:01,TT,2023-10-06
1,850867434:01,465.9,2023-12-17
2,850867492:01,308.3,2023-06-17
3,850867492:01,308.3,2023-06-18
4,850867492:01,300.00,2023-10-19


In [3]:
        
#Call and apply the function in Step 1
final = transform_df(members,claims_df,elig,yellow_members)


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

columns  Index(['member_id', 'market_most_visits', 'total_visits_count',
       'eligible_yrmth_count', 'member_location', 'member_age',
       'member_gender', 'IP_visits_count', 'OP_visits_count',
       'IP_total_paid_amount', 'OP_total_paid_amount',
       'yf249_Type1Diabetes_indicator', 'yf249_Type1Diabetes_visits_count',
       'yf249_Type1Diabetes_visits_rate', 'yf250_Type2Diabetes_indicator',
       'yf250_Type2Diabetes_visits_count', 'yf250_Type2Diabetes_visits_rate',
       'yf272_Dyslipidemia_indicator', 'yf272_Dyslipidemia_visits_count',
       'yf272_Dyslipidemia_visits_rate', 'yf401_Hypertension_indicator',
       'yf401_Hypertension_visits_count', 'yf401_Hypertension_visits_rate',
       'yf305_TobaccoUse_indicator', 'yf305_TobaccoUse_visits_count',
       'yf305_TobaccoUse_visits_rate', 'yf3055_illicitDrugUse_indicator',
       'yf3055_illicitDrugUse_visits_count', 'yf311_Depression_indicator',
       'yf311_Depression_visits_count', 'yf311_Depression_visits_rate',
   

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:
final.dtypes

member_age                                                             int64
IP_total_visits_rate                                                 float64
OP_total_visits_rate                                                 float64
yf249_Type1Diabetes_visits_rate                                      float64
yf250_Type2Diabetes_visits_rate                                      float64
yf272_Dyslipidemia_visits_rate                                       float64
yf401_Hypertension_visits_rate                                       float64
yf305_TobaccoUse_visits_rate                                         float64
yf311_Depression_visits_rate                                         float64
yf300_Anxiety_visits_rate                                            float64
yf955_Polypharmcy_visits_rate                                        float64
yf309_Acopia_visits_rate                                             float64
yf278_Obesity_visits_rate                                            float64

In [5]:
from dateutil.relativedelta import relativedelta

In [8]:
##run date
latest_date_name = claims_df["incurred__date__year_month_"].max()

 
# 3 months before the latest date of data (index date)
index_date = claims_df["incurred__date__year_month_"].max() - relativedelta(months=3)

# 1 year before index date (label the final csv)
year_before_index_date_name = (claims_df["incurred__date__year_month_"].max()- relativedelta(months=3))- relativedelta(months=12) 

# 3 year ago from the latest date of data
three_years_ago = claims_df["incurred__date__year_month_"].max() - relativedelta(months=36)

# the oldest date got from the source (4-years ago)
four_years_ago = claims_df["incurred__date__year_month_"].max() - relativedelta(months=48)

In [9]:
# Observation period
obs_start_date=claims_df['incurred__date__year_month_'].max()-pd.DateOffset(months=15) 
obs_end_date=claims_df['incurred__date__year_month_'].max()-pd.DateOffset(months=3) 

In [10]:
# performance period:
per_start_date=claims_df['incurred__date__year_month_'].max()-pd.DateOffset(months=3)+pd.DateOffset(days=1) 
per_end_date=claims_df['incurred__date__year_month_'].max()

In [11]:
print("Number of total Claim records in member level: " + str(len(final.index)))
print("The latest date got from the source: " + str(latest_date_name))
print("The index_date: " + str(index_date))
print("1 year before index date: " + str(year_before_index_date_name))
print("3 year ago from the latest date of data: " + str(three_years_ago))
print("max data we pulled (4 years ago): " + str(four_years_ago))

Number of total Claim records in member level: 9227
The latest date got from the source: 2023-12-28 00:00:00
The index_date: 2023-09-28 00:00:00
1 year before index date: 2022-09-28 00:00:00
3 year ago from the latest date of data: 2020-12-28 00:00:00
max data we pulled (4 years ago): 2019-12-28 00:00:00


In [12]:
print("Observation start date: " + str (obs_start_date))
print("observation end date: " + str(obs_end_date))
print("performance start date: " + str(per_start_date))
print("performance end date: " + str(per_end_date))

Observation start date: 2022-09-28 00:00:00
observation end date: 2023-09-28 00:00:00
performance start date: 2023-09-29 00:00:00
performance end date: 2023-12-28 00:00:00


In [13]:
final.head(5)

,member_age,IP_total_visits_rate,OP_total_visits_rate,yf249_Type1Diabetes_visits_rate,yf250_Type2Diabetes_visits_rate,yf272_Dyslipidemia_visits_rate,yf401_Hypertension_visits_rate,yf305_TobaccoUse_visits_rate,yf311_Depression_visits_rate,yf300_Anxiety_visits_rate,yf955_Polypharmcy_visits_rate,yf309_Acopia_visits_rate,yf278_Obesity_visits_rate,yf3002_Kinesiophobia_visits_rate,IP_total_paid_amount_pppm,OP_total_paid_amount_pppm,physiotherapy_visits_rate,surgery_visits_rate,radio_diag_scans_visits_rate,eval_manag_services_visits_rate,alternative_medicine_visits_rate,drugs_related_visits_rate,anesthesia_visits_rate,all_maternity_indicator,member_gender_F,member_gender_M,member_family_type_C,member_family_type_D,member_family_type_F,member_family_type_S,market_latest_index_market1,market_latest_index_market2,member_industry_Banking_Investment_&_Insurance_inc_Brokers_Agent_,member_industry_Chemicals_Petroleum_Refining_&_Assoc_Products,member_industry_Communications_&_Computing,member_industry_Community_&_Social_Services,member_industry_Consultancy_Marketing_&_Legal_services,member_industry_Dev_Buying_&_Selling_of_Domestic_or_Comm_Property,member_industry_Education,member_industry_Electricity_&_Gas_Supply_Water_Sewage_&_Drainage,member_industry_Food_Textiles_and_Retailing,member_industry_Freight_Transportation_&_Warehousing,member_industry_Government_Administration,member_industry_Health,member_industry_Hotels_&_Restaurants,member_industry_Light_Electrical_&_Precision_Manufacturing,member_industry_Manufacturing_and_Retail,member_industry_Mining_&_Quarrying_incl_Oil_&_Gas_Extraction_,member_industry_Miscellaneous,member_industry_Transportation_and_Services,member_industry_Wholesale,member_location_region_AFRICA,member_location_region_AMERICAS,member_location_region_ASIA,member_location_region_EUROPE,member_location_region_OCEANIA
member_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
850004357:01,57,0.0,1.75,0.0,0.0,0.0,0.0,0.0,0.750000,0.625,0.125000,0.0,0.625000,0.75,541.396209,541.396209,0.000000,0.000000,250.000000,1000.0,375.0,500.0,0.0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
850004357:02,56,0.0,4.75,0.0,0.0,0.0,0.0,0.0,1.666667,1.500,0.166667,0.0,5.083333,1.50,1204.646825,1204.646825,2333.333333,83.333333,83.333333,1000.0,500.0,500.0,0.0,1,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
850004965:01,58,0.0,1.20,0.0,0.0,0.0,0.0,0.0,0.600000,0.000,1.000000,0.0,2.800000,1.60,385.224000,385.224000,0.000000,0.000000,200.000000,400.0,0.0,1200.0,0.0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
850012059:02,47,0.0,1.60,0.0,0.0,0.0,0.0,0.0,0.000000,0.000,4.400000,0.0,3.400000,0.00,266.428000,266.428000,800.000000,0.000000,0.000000,1000.0,0.0,0.0,0.0,0,1,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
850143894:01,38,0.0,2.40,0.0,0.9,0.0,0.0,0.0,0.400000,0.300,0.100000,0.0,3.400000,3.00,459.869000,459.869000,200.000000,0.000000,100.000000,700.0,500.0,1000.0,0.0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0


In [ ]:
# # Automate to generate the latest csv for modeling
# wr.s3.to_csv(
#     df=final,
#     path='s3://intl-asia-hk-datascientist-prod-bucket/msk_global_pilot/MSK_global_SG_HK_preprocessing_index_20231228.csv') 

# To do- path for parquet file?

In [ ]:
# df = transform_df(members,claims_df,elig,yellow_members)


# output_path = os.path.join('/opt/ml/processing/output', 'output.parquet')
# df.to_parquet(output_path,
#                coerce_timestamps='ms',
#                engine="pyarrow")